In [21]:
import sys
sys.path.append("..")
from lib.util import add_race_eth,  add_income_group
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1550
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas

In [22]:
add_race_eth(df)
dfs = df[['id','census_tract_number', 'applicant_ethnicity','applicant_race_1', 'race_eth']]

In [23]:
ts = l.get('ffiec.gov-hmda-2010e-ts').partition.pandas
ts_name = ts[['respondent_id','respondent_name']]
x = df[['id','respondent_id', 'loan_type']].merge(ts_name, on='respondent_id')
x[x.respondent_name.str.contains('CITIBANK')][['respondent_id','respondent_name']].drop_duplicates()

,respondent_id,respondent_name
247377,0000476810,"CITIBANK, N.A."


# CitiBank
- **Missing: Minority Status, Tract/BNA**

In [24]:
df = df[df.respondent_id=='0000476810']

###
# Action type == ALL
###
totalcount = df.loan_amount.count()
totalamount = df.loan_amount.sum()

### By Loan Purpose and Type

df['loan_purtype'] = '_'
df['loan_purtype'][(df['loan_purpose'] == 1) & (df['loan_type'] == 1)] = 'Purchase - Conventional'
df['loan_purtype'][(df['loan_purpose'] == 1) & (df['loan_type'] > 1)] = 'Purchase - Goverment'
df['loan_purtype'][df['loan_purpose'] == 2] = 'Home Improvement'
df['loan_purtype'][df['loan_purpose'] == 3] = 'Refinancing'

loanpt = df.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df['applicant_income_percent_median'] = 100*df.applicant_income/df.applicant_income.median()
df['applicant_income_cat'] = 'Income Not Available'
df['applicant_income_cat'][df['applicant_income_percent_median'] < 50] = 'Low'
df['applicant_income_cat'][(df['applicant_income_percent_median'] >= 50) & (df['applicant_income_percent_median'] < 80)] = 'Moderate'
df['applicant_income_cat'][(df['applicant_income_percent_median'] >= 80) & (df['applicant_income_percent_median'] < 120)] = 'Middle'
df['applicant_income_cat'][df['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
totalapp
pieces = [totalapp.ix[:, ["count","%count","amount","%amount",]]]
totalapp = pd.concat(pieces, axis=1, keys=['Total Applications (1)'])
totalapp

Total Applications (1)          \
                                                                count  %count   
Loan Purpose and Type Home Improvement                             56     1.0   
                      Purchase - Conventional                     285     5.3   
                      Purchase - Goverment                         29     0.5   
                      Refinancing                                5010    93.1   
Applicant Race/Eth    aian                                         23     0.4   
                      as                                          429     8.0   
                      b                                            70     1.3   
                      h                                           433     8.0   
                      pi                                           42     0.8   
                      w                                          2506    46.6   
Applicant Income      Income Not Available                         68     1.3   
                      Low                                         846    15.7   
                      Middle                                     1334    24.8   
                      Moderate                                   1066    19.8   
                      Upper                                      2066    38.4   
Applicant Sex         1                                          2769    51.5   
                      2                                          1094    20.3   
                      3                                           659    12.2   
                      4                                           858    15.9   

                                                                 
                                                amount  %amount  
Loan Purpose and Type Home Improvement           15728      0.9  
                      Purchase - Conventional   154893      9.3  
                      Purchase - Goverment        9760      0.6  
                      Refinancing              1479116     89.1  
Applicant Race/Eth    aian                        5267      0.3  
                      as                        127684      7.7  
                      b                          16777      1.0  
                      h                          94641      5.7  
                      pi                          9502      0.6  
                      w                         711225     42.9  
Applicant Income      Income Not Available       40544      2.4  
                      Low                       155109      9.3  
                      Middle                    354045     21.3  
                      Moderate                  233620     14.1  
                      Upper                     876179     52.8  
Applicant Sex         1                         851504     51.3  
                      2                         285760     17.2  
                      3                         244960     14.8  
                      4                         277273     16.7

In [25]:
#######
# Action Type =1
#######
df1 = df[df.action_type==1]
totalcount = df1.loan_amount.count()
totalamount = df1.loan_amount.sum()

### By Loan Purpose and Type
df1['loan_purtype'] = '_'
df1['loan_purtype'][(df1['loan_purpose'] == 1) & (df1['loan_type'] == 1)] = 'Purchase - Conventional'
df1['loan_purtype'][(df1['loan_purpose'] == 1) & (df1['loan_type'] > 1)] = 'Purchase - Goverment'
df1['loan_purtype'][df1['loan_purpose'] == 2] = 'Home Improvement'
df1['loan_purtype'][df1['loan_purpose'] == 3] = 'Refinancing'

loanpt = df1.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df1.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]


###By Applicant Income

df1['applicant_income_percent_median'] = 100*df1.applicant_income/df1.applicant_income.median()
df1['applicant_income_cat'] = 'Income Not Available'
df1['applicant_income_cat'][df1['applicant_income_percent_median'] < 50] = 'Low'
df1['applicant_income_cat'][(df1['applicant_income_percent_median'] >= 50) & (df1['applicant_income_percent_median'] < 80)] = 'Moderate'
df1['applicant_income_cat'][(df1['applicant_income_percent_median'] >= 80) & (df1['applicant_income_percent_median'] < 120)] = 'Middle'
df1['applicant_income_cat'][df1['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df1.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df1.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp1 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp1.ix[:, ["count","%count","amount","%amount",]]]
totalapp1 = pd.concat(pieces, axis=1, keys=['Loan originated'])
totalapp1

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


Loan originated          \
                                                         count  %count   
Loan Purpose and Type Home Improvement                      17     0.7   
                      Purchase - Conventional              124     5.2   
                      Purchase - Goverment                   8     0.3   
                      Refinancing                         2253    93.8   
Applicant Race/Eth    aian                                  12     0.5   
                      as                                   227     9.5   
                      b                                     37     1.5   
                      h                                    224     9.3   
                      pi                                    22     0.9   
                      w                                   1365    56.8   
Applicant Income      Income Not Available                  20     0.8   
                      Low                                  402    16.7   
                      Middle                               558    23.2   
                      Moderate                             471    19.6   
                      Upper                                951    39.6   
Applicant Sex         1                                   1494    62.2   
                      2                                    566    23.6   
                      3                                    339    14.1   
                      4                                      3     0.1   

                                                                
                                               amount  %amount  
Loan Purpose and Type Home Improvement           6508      0.9  
                      Purchase - Conventional   88649     11.9  
                      Purchase - Goverment       2302      0.3  
                      Refinancing              647027     86.9  
Applicant Race/Eth    aian                       3037      0.4  
                      as                        71735      9.6  
                      b                          8937      1.2  
                      h                         49783      6.7  
                      pi                         5628      0.8  
                      w                        384219     51.6  
Applicant Income      Income Not Available      24038      3.2  
                      Low                       70742      9.5  
                      Middle                   140671     18.9  
                      Moderate                  99818     13.4  
                      Upper                    409217     55.0  
Applicant Sex         1                        467880     62.8  
                      2                        138704     18.6  
                      3                        119702     16.1  
                      4                         18200      2.4

In [26]:
#######
# Action Type =2 
#######
df2 = df[df.action_type==2]
totalcount = df2.loan_amount.count()
totalamount = df2.loan_amount.sum()

### By Loan Purpose and Type

df2['loan_purtype'] = '_'
df2['loan_purtype'][(df2['loan_purpose'] == 1) & (df2['loan_type'] == 1)] = 'Purchase - Conventional'
df2['loan_purtype'][(df2['loan_purpose'] == 1) & (df2['loan_type'] > 1)] = 'Purchase - Goverment'
df2['loan_purtype'][df2['loan_purpose'] == 2] = 'Home Improvement'
df2['loan_purtype'][df2['loan_purpose'] == 3] = 'Refinancing'

loanpt = df2.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df2.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df2['applicant_income_percent_median'] = 100*df2.applicant_income/df2.applicant_income.median()
df2['applicant_income_cat'] = 'Income Not Available'
df2['applicant_income_cat'][df2['applicant_income_percent_median'] < 50] = 'Low'
df2['applicant_income_cat'][(df2['applicant_income_percent_median'] >= 50) & (df2['applicant_income_percent_median'] < 80)] = 'Moderate'
df2['applicant_income_cat'][(df2['applicant_income_percent_median'] >= 80) & (df2['applicant_income_percent_median'] < 120)] = 'Middle'
df2['applicant_income_cat'][df2['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df2.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df2.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp2 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp2.ix[:, ["count","%count","amount","%amount",]]]
totalapp2 = pd.concat(pieces, axis=1, keys=['App approved but not accepted'])
totalapp2

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


App approved but not accepted  \
                                                                       count   
Loan Purpose and Type Purchase - Conventional                              2   
                      Refinancing                                         34   
Applicant Race/Eth    as                                                   2   
                      h                                                    3   
                      pi                                                   1   
                      w                                                   22   
Applicant Income      Low                                                  4   
                      Middle                                              10   
                      Moderate                                            10   
                      Upper                                               12   
Applicant Sex         1                                                   23   
                      2                                                    9   
                      3                                                    4   

                                                                        
                                               %count  amount  %amount  
Loan Purpose and Type Purchase - Conventional     5.6    1500     15.1  
                      Refinancing                94.4    8412     84.9  
Applicant Race/Eth    as                          5.6     453      4.6  
                      h                           8.3     786      7.9  
                      pi                          2.8     243      2.5  
                      w                          61.1    5696     57.5  
Applicant Income      Low                        11.1     701      7.1  
                      Middle                     27.8    2602     26.3  
                      Moderate                   27.8    2232     22.5  
                      Upper                      33.3    4377     44.2  
Applicant Sex         1                          63.9    5821     58.7  
                      2                          25.0    2285     23.1  
                      3                          11.1    1806     18.2

In [27]:
#######
# Action Type =3 
#######
df3 = df[df.action_type==3]
totalcount = df3.loan_amount.count()
totalamount = df3.loan_amount.sum()

### By Loan Purpose and Type

df3['loan_purtype'] = '_'
df3['loan_purtype'][(df3['loan_purpose'] == 1) & (df3['loan_type'] == 1)] = 'Purchase - Conventional'
df3['loan_purtype'][(df3['loan_purpose'] == 1) & (df3['loan_type'] > 1)] = 'Purchase - Goverment'
df3['loan_purtype'][df3['loan_purpose'] == 2] = 'Home Improvement'
df3['loan_purtype'][df3['loan_purpose'] == 3] = 'Refinancing'

loanpt = df3.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df3.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df3['applicant_income_percent_median'] = 100*df3.applicant_income/df3.applicant_income.median()
df3['applicant_income_cat'] = 'Income Not Available'
df3['applicant_income_cat'][df3['applicant_income_percent_median'] < 50] = 'Low'
df3['applicant_income_cat'][(df3['applicant_income_percent_median'] >= 50) & (df3['applicant_income_percent_median'] < 80)] = 'Moderate'
df3['applicant_income_cat'][(df3['applicant_income_percent_median'] >= 80) & (df3['applicant_income_percent_median'] < 120)] = 'Middle'
df3['applicant_income_cat'][df3['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df3.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df3.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp3 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp3.ix[:, ["count","%count","amount","%amount",]]]
totalapp3 = pd.concat(pieces, axis=1, keys=['App denied by financial institution'])
totalapp3

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


App denied by financial institution  \
                                                                             count   
Loan Purpose and Type Home Improvement                                           6   
                      Purchase - Conventional                                   27   
                      Purchase - Goverment                                       2   
                      Refinancing                                              692   
Applicant Race/Eth    aian                                                       5   
                      as                                                        81   
                      b                                                         16   
                      h                                                         66   
                      pi                                                         5   
                      w                                                        399   
Applicant Income      Income Not Available                                      17   
                      Low                                                      121   
                      Middle                                                   155   
                      Moderate                                                 154   
                      Upper                                                    280   
Applicant Sex         1                                                        428   
                      2                                                        189   
                      3                                                        110   

                                                                        
                                               %count  amount  %amount  
Loan Purpose and Type Home Improvement            0.8    1554      0.7  
                      Purchase - Conventional     3.7   11874      5.4  
                      Purchase - Goverment        0.3     637      0.3  
                      Refinancing                95.2  206473     93.6  
Applicant Race/Eth    aian                        0.7     857      0.4  
                      as                         11.1   20397      9.2  
                      b                           2.2    3615      1.6  
                      h                           9.1   14448      6.6  
                      pi                          0.7     988      0.4  
                      w                          54.9  113393     51.4  
Applicant Income      Income Not Available        2.3    5733      2.6  
                      Low                        16.6   27091     12.3  
                      Middle                     21.3   42328     19.2  
                      Moderate                   21.2   32386     14.7  
                      Upper                      38.5  113000     51.2  
Applicant Sex         1                          58.9  121345     55.0  
                      2                          26.0   57448     26.0  
                      3                          15.1   41745     18.9

In [28]:
#######
# Action Type =4
#######
df4 = df[df.action_type==4]
totalcount = df4.loan_amount.count()
totalamount = df4.loan_amount.sum()

### By Loan Purpose and Type

df4['loan_purtype'] = '_'
df4['loan_purtype'][(df4['loan_purpose'] == 1) & (df4['loan_type'] == 1)] = 'Purchase - Conventional'
df4['loan_purtype'][(df4['loan_purpose'] == 1) & (df4['loan_type'] > 1)] = 'Purchase - Goverment'
df4['loan_purtype'][df4['loan_purpose'] == 2] = 'Home Improvement'
df4['loan_purtype'][df4['loan_purpose'] == 3] = 'Refinancing'

loanpt = df4.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df4.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df4['applicant_income_percent_median'] = 100*df4.applicant_income/df4.applicant_income.median()
df4['applicant_income_cat'] = 'Income Not Available'
df4['applicant_income_cat'][df4['applicant_income_percent_median'] < 50] = 'Low'
df4['applicant_income_cat'][(df4['applicant_income_percent_median'] >= 50) & (df4['applicant_income_percent_median'] < 80)] = 'Moderate'
df4['applicant_income_cat'][(df4['applicant_income_percent_median'] >= 80) & (df4['applicant_income_percent_median'] < 120)] = 'Middle'
df4['applicant_income_cat'][df4['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df4.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df4.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp4 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp4.ix[:, ["count","%count","amount","%amount",]]]
totalapp4 = pd.concat(pieces, axis=1, keys=['App withdrawn by applicant'])
totalapp4

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


App withdrawn by applicant  \
                                                                    count   
Loan Purpose and Type Home Improvement                                  7   
                      Purchase - Conventional                          39   
                      Refinancing                                     647   
Applicant Race/Eth    aian                                              3   
                      as                                               64   
                      b                                                 8   
                      h                                                58   
                      pi                                                6   
                      w                                               384   
Applicant Income      Income Not Available                             12   
                      Low                                             102   
                      Middle                                          156   
                      Moderate                                        145   
                      Upper                                           278   
Applicant Sex         1                                               408   
                      2                                               177   
                      3                                               108   

                                                                        
                                               %count  amount  %amount  
Loan Purpose and Type Home Improvement            1.0    1581      0.7  
                      Purchase - Conventional     5.6   22853     10.2  
                      Refinancing                93.4  199498     89.1  
Applicant Race/Eth    aian                        0.4     759      0.3  
                      as                          9.2   20958      9.4  
                      b                           1.2    1813      0.8  
                      h                           8.4   12695      5.7  
                      pi                          0.9    1088      0.5  
                      w                          55.4  116240     51.9  
Applicant Income      Income Not Available        1.7    3457      1.5  
                      Low                        14.7   17764      7.9  
                      Middle                     22.5   41834     18.7  
                      Moderate                   20.9   31389     14.0  
                      Upper                      40.1  129488     57.8  
Applicant Sex         1                          58.9  133526     59.6  
                      2                          25.5   46455     20.7  
                      3                          15.6   43951     19.6

In [29]:
#######
# Action Type =5
#######

df5 = df[df.action_type==5]
totalcount = df5.loan_amount.count()
totalamount = df5.loan_amount.sum()

### By Loan Purpose and Type

df5['loan_purtype'] = '_'
df5['loan_purtype'][(df5['loan_purpose'] == 1) & (df5['loan_type'] == 1)] = 'Purchase - Conventional'
df5['loan_purtype'][(df5['loan_purpose'] == 1) & (df5['loan_type'] > 1)] = 'Purchase - Goverment'
df5['loan_purtype'][df5['loan_purpose'] == 2] = 'Home Improvement'
df5['loan_purtype'][df5['loan_purpose'] == 3] = 'Refinancing'

loanpt = df5.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df5.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df5['applicant_income_percent_median'] = 100*df5.applicant_income/df5.applicant_income.median()
df5['applicant_income_cat'] = 'Income Not Available'
df5['applicant_income_cat'][df5['applicant_income_percent_median'] < 50] = 'Low'
df5['applicant_income_cat'][(df5['applicant_income_percent_median'] >= 50) & (df5['applicant_income_percent_median'] < 80)] = 'Moderate'
df5['applicant_income_cat'][(df5['applicant_income_percent_median'] >= 80) & (df5['applicant_income_percent_median'] < 120)] = 'Middle'
df5['applicant_income_cat'][df5['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df5.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df5.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp5 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp5.ix[:, ["count","%count","amount","%amount",]]]
totalapp5 = pd.concat(pieces, axis=1, keys=['File closed for incompleteness'])
totalapp5

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:39: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


File closed for incompleteness  \
                                                                        count   
Loan Purpose and Type Home Improvement                                     11   
                      Purchase - Conventional                              11   
                      Purchase - Goverment                                  2   
                      Refinancing                                         643   
Applicant Race/Eth    aian                                                  3   
                      as                                                   55   
                      b                                                     9   
                      h                                                    82   
                      pi                                                    8   
                      w                                                   336   
Applicant Income      Income Not Available                                  7   
                      Low                                                 117   
                      Middle                                              177   
                      Moderate                                            117   
                      Upper                                               249   
Applicant Sex         1                                                   416   
                      2                                                   153   
                      3                                                    98   

                                                                        
                                               %count  amount  %amount  
Loan Purpose and Type Home Improvement            1.6    2504      1.2  
                      Purchase - Conventional     1.6    5939      2.9  
                      Purchase - Goverment        0.3     477      0.2  
                      Refinancing                96.4  192636     95.6  
Applicant Race/Eth    aian                        0.4     614      0.3  
                      as                          8.2   14141      7.0  
                      b                           1.3    2412      1.2  
                      h                          12.3   16929      8.4  
                      pi                          1.2    1555      0.8  
                      w                          50.4   91677     45.5  
Applicant Income      Income Not Available        1.0    2534      1.3  
                      Low                        17.5   20081     10.0  
                      Middle                     26.5   43521     21.6  
                      Moderate                   17.5   23914     11.9  
                      Upper                      37.3  111506     55.3  
Applicant Sex         1                          62.4  122932     61.0  
                      2                          22.9   40868     20.3  
                      3                          14.7   37756     18.7

In [30]:
#######
# Action Type = 6
#######

df6 = df[df.action_type==6]
totalcount = df6.loan_amount.count()
totalamount = df6.loan_amount.sum()

### By Loan Purpose and Type

df6['loan_purtype'] = '_'
df6['loan_purtype'][(df6['loan_purpose'] == 1) & (df6['loan_type'] == 1)] = 'Purchase - Conventional'
df6['loan_purtype'][(df6['loan_purpose'] == 1) & (df6['loan_type'] > 1)] = 'Purchase - Goverment'
df6['loan_purtype'][df6['loan_purpose'] == 2] = 'Home Improvement'
df6['loan_purtype'][df6['loan_purpose'] == 3] = 'Refinancing'

loanpt = df6.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df6.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df6['applicant_income_percent_median'] = 100*df6.applicant_income/df6.applicant_income.median()
df6['applicant_income_cat'] = 'Income Not Available'
df6['applicant_income_cat'][df6['applicant_income_percent_median'] < 50] = 'Low'
df6['applicant_income_cat'][(df6['applicant_income_percent_median'] >= 50) & (df6['applicant_income_percent_median'] < 80)] = 'Moderate'
df6['applicant_income_cat'][(df6['applicant_income_percent_median'] >= 80) & (df6['applicant_income_percent_median'] < 120)] = 'Middle'
df6['applicant_income_cat'][df6['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df6.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df6.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together
frames = [loanpt, race_eth,income,sex]
totalapp6 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp6.ix[:, ["count","%count","amount","%amount",]]]
totalapp6 = pd.concat(pieces, axis=1, keys=['Loan purchased by institution'])
totalapp6

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:39: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


Loan purchased by institution  \
                                                                       count   
Loan Purpose and Type Home Improvement                                    15   
                      Purchase - Conventional                             82   
                      Purchase - Goverment                                17   
                      Refinancing                                        741   
Applicant Income      Income Not Available                                12   
                      Low                                                 83   
                      Middle                                             268   
                      Moderate                                           200   
                      Upper                                              292   
Applicant Sex         4                                                  855   

                                                                        
                                               %count  amount  %amount  
Loan Purpose and Type Home Improvement            1.8    3581      1.4  
                      Purchase - Conventional     9.6   24078      9.3  
                      Purchase - Goverment        2.0    6344      2.4  
                      Refinancing                86.7  225070     86.9  
Applicant Income      Income Not Available        1.4    4782      1.8  
                      Low                         9.7   16676      6.4  
                      Middle                     31.3   82831     32.0  
                      Moderate                   23.4   52648     20.3  
                      Upper                      34.2  102136     39.4  
Applicant Sex         4                         100.0  259073    100.0

In [ ]:
#Merging All

In [32]:
#citi.to_csv('citi.csv')